# Gene Regulatory Network Inference
This is the notebook to process data and define functions before making a python script

In [1]:
# Import basic packages
import pandas as pd
import numpy as np

In [26]:
# LCL expression data
lcl_exp_path = '../dataset/LCL_networks/expression'
lcl_geu = pd.read_csv(f'{lcl_exp_path}/Geuvadis.txt', sep = '\t').set_index('Gene')

In [107]:
# Make prepare dataset function
from sklearn import model_selection
def prep_dataset(target_gene, exp_df):
    '''
    Prepares training set and test set for target gene
    
    Args:
        - target_gene: target gene for the iteration (y)
        - exp_df: expression dataframe (already in pandas df format)
        
    Returns:
        - Training and Testing set to be used in model predictions
    '''
    # Get y (target) and predictor matrix (X)
    y = exp_df.loc[target_gene, :]
    X = exp_df.drop(target_gene).transpose().values
    
    
    # Split 80:20 for test and train
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.2)
    
    return X_train, X_test, y_train, y_test

In [113]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

def grn_lasso(target_gene, exp_df):
    '''
    GRN inference method using lasso regression
    
    Args:
        - target_gene: target gene for the iteration (y)
        - exp_df: expression dataframe (already in pandas df format)
        
    Returns:
        - Numpy array of type str, with list of non-zero weight predictors
    '''
    # Prep data
    X_train, X_test, y_train, y_test = prep_dataset(target_gene, exp_df)
    
    # Use Lasso regression
    lasso_reg = Lasso(alpha = 0.1)
    lasso_reg.fit(X_train, y_train)
    
    # Get scores (R^2)
    train_score = lasso_reg.score(X_train, y_train) # Note: R^2 not very good, maybe use other methods
    test_score = lasso_reg.score(X_test, y_test)
    
    # Get weights of lasso, non zero weights are regulators
    predictors = exp_df.drop(target_gene).index
    nonzero_filter = lasso_reg.coef_ != 0
    nonzero_preds = predictors[nonzero_filter]
    
    
    return nonzero_preds.values

def grn_regforest(target_gene, exp_df):
    '''
    GRN inference method using regression forest. This method does not assume linearity of data.
    
    Args:
        - target_gene: target gene for the iteration (y)
        - exp_df: expression dataframe (already in pandas df format)
        
    Returns:
        - Numpy array of type str, with list of non-zero weight predictors
        
    '''
    # Prep data
    X_train, X_test, y_train, y_test = prep_dataset(target_gene, exp_df)
    
    # Use regerssion tree
    forest_reg = RandomForestRegressor(n_estimators = 10, max_depth = 8, bootstrap = True, min_samples_leaf = 10, n_jobs=-1)
    forest_reg.fit(X_train, y_train)
    
    # Get Scores (R^2)
    train_score = forest_reg.score(X_train, y_train)
    test_score = forest_reg.score(X_test, y_test)
    
    # Get feature importance
    predictors = exp_df.drop(target_gene).index
    nonzero_filter = forest_reg.feature_importances_ != 0
    nonzero_preds = predictors[nonzero_filter]
    
    return nonzero_preds.values



In [109]:
# Read in an expression file, and predict GRN using a method
def predict_grn(exp_file, method, *args):
    '''
    Function that reads in expression level and predict its GRN using a specific method
    
    Args:
        - exp_file: Path to expression file
        - method: Method of inferring GRN (Example: grn_lasso)
        - *args: additional arguments for method
    
    Returns:
        - Set of predicted edges with format (regulator(space)target)
    '''

    exp_df = pd.read_csv(exp_file, sep = '\t').set_index('Gene')

    pred_edges = set()
    
    
    for target in exp_df.index.values:
        # Get regulators for every target gene
        pred_regs = method(target, exp_df, *args)
        
        # Add the predicted regulators -> target edge to the predicted edges
        for reg in pred_regs:
            pred_edges.add(f'{reg}->{target}')
    
    return pred_edges

In [114]:
# Performance Metrics

def iou_score(gold_file, pred_grn_edges):
    '''
    Function to score predicted grn vs gold standard based on intersection over union.

    Score:
    For starters, we will use the simple score of intersection / union
    Intersection: Count of edges in both gold standard AND predicted grn
    Union: Count of gold standard edges + predicted grn edges - Intersection

    Intersection / Union is the score.

    Args:
        - gold_file: Path to gold standard file
        - pred_grn_edges: set of predicted grn edges

    Returns:
        - IOU score
    '''

    # Read in gold standard file
    gold_df = pd.read_csv(gold_file, sep = '\t', header = None, names = ['Regulator', 'Target']) 

    # Set of gold standard edges
    gold_edge_set = set(gold_df.loc[:, 'Regulator'] + '->' + gold_df.loc[:, 'Target'])

    # Get Intersection and Union
    intersection = gold_edge_set.intersection(pred_grn_edges)
    union = gold_edge_set.union(pred_grn_edges)

    # Get score: Intersection / Union
    iou_score = len(intersection) / len(union)
    print(f'Union edges count: {len(union)}')
    print(f'Intersection edges count: {len(intersection)}')
    
    return iou_score

In [115]:
# Compare scores between methods vs gold standard
lcl_gold = '../dataset/LCL_networks/gold/Cusanovich_gold.txt'
lcl_geu_path = f'{lcl_exp_path}/Geuvadis.txt'

# Predict using each methods
lcl_lasso_pred = predict_grn(lcl_geu_path, grn_lasso)
lcl_regforest_pred = predict_grn(lcl_geu_path, grn_regforest)


Union edges count: 34734
Intersection edges count: 2
Union edges count: 621345
Intersection edges count: 230
IOU Score, lasso: 5.758046870501526e-05
IOU Score, regression random forest: 0.00037016472330186933


In [116]:
# Get iou scores
lcl_lasso_iou = iou_score(lcl_gold, lcl_lasso_pred)
lcl_regforest_iou = iou_score(lcl_gold, lcl_regforest_pred)

print(f'IOU Score, lasso: {lcl_lasso_iou}')
print(f'IOU Score, regression random forest: {lcl_regforest_iou}')

Union edges count: 34734
Intersection edges count: 2
Union edges count: 621345
Intersection edges count: 230
IOU Score, lasso: 5.758046870501526e-05
IOU Score, regression random forest: 0.00037016472330186933
